In [11]:
from google.colab import drive
drive.mount('/content/drive')
import csv
import re
import random
import tqdm.notebook as tq
import pandas as pd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 讀入語料與構式辭典

In [12]:
file_path = '/content/drive/Shareddrives/LOPE/產學合作/中華電信/2020-2021/指向情緒案/data/threads/cht-2020-merged.txt'

with open(file_path, 'r', encoding = 'utf-8') as f:
  comments = [ line.strip() for line in f ] # comments 中的每一個 element 是一則評論

In [14]:
construcion_path = '/content/drive/Shareddrives/LOPE/產學合作/中華電信/2020-2021/指向情緒案/sentiment-construction-list/constructions_list/constructions_0406.csv'

with open(construcion_path) as f:
    constructions = [ row.strip() for row in f ]

## 從每則評論中找出符合的構式例子

In [47]:
n = 5  # sample size
random.seed(10)

matched = {}
matched_con = {}
for k in tq.tqdm(constructions):
  pat = re.compile(k)
  if k not in matched: matched[k] = []
  
  # Get all cnstr from all comments
  candidates = set()
  for cmt in comments:
    for c in pat.finditer(cmt): candidates.add(c[0])
  
  # Count all cnstr
  matched_con[k] = len(candidates)

  # Sample
  if len(candidates) > n:
    for c in random.sample(candidates, n): matched[k].append(c)
  else:
    for c in candidates: matched[k].append(c)

In [18]:
import pprint
pprint.pprint(matched)

{'(對\\w+)?一點(都|也)不\\w+\\b': ['一點都不受影響',
                             '一點都不想再多花400元呢',
                             '一點都不奇怪啊',
                             '一點都不會後悔',
                             '一點也不過分'],
 '(很|也)夠\\w+了\\b': ['也夠打94分鐘了', '也夠鄉下了', '也夠我用了', '很夠很夠用了', '很夠力了'],
 '(真的)?有夠\\w+的?\\b': ['有夠難看',
                      '有夠大昨天有小豬說台廠已經出局',
                      '有夠爛的遠傳',
                      '有夠麻煩其他平台都有',
                      '有夠爛的中華電信'],
 '(置|拉)板凳等\\w+\\b': ['拉板凳等中華', '置板凳等後續'],
 '\\b\\w+(到|的|得)離譜': ['難以置信的離譜',
                      '藉此可打亂一下三大龍頭的離譜',
                      '寫的資料錯的離譜',
                      '慢的離譜',
                      '5g標金高得離譜'],
 '\\b\\w+(掰|bye)了': ['line帳號就掰了',
                     '彰銀my樂應該真的掰了',
                     '新增消費5000門檻就bye了',
                     '有前綴代扣字樣就bye了',
                     '就等四月合約到期可以掰了'],
 '\\b\\w+(的|得)要死': ['訊號爛的要死',
                    '結果最近網路慢的要死',
                    '客服人員態度還差得要死',
                    '笑死晚上看影片卡的要死',
                  

### 將符合構式的例子及其數量整理成表格

In [46]:
df = pd.DataFrame([matched_con, matched]).T
df.columns = ['count', 'examples']
df

,count,examples
只有遠傳\s?沒有\w{2},25,"[只有遠傳沒有收訊, 只有遠傳沒有訊息, 只有遠傳 沒有ｏｏ, 只有遠傳 沒有消息, 只有遠..."
純噓\w+\b,20,"[純噓亞太, 純噓頂新之心, 純噓獲利王, 純噓樓上, 純噓林之晨]"
\b\w+能\w\s?，?\w+都能\w了\b,2,[]
八成\w+\b,81,"[八成蠻適合習慣看第四台的人, 八成是app有要求業務下載量y, 八成人口涵蓋率, 八成又是..."
\b\w+秀下限\w+\b,16,"[厚顏無恥集體秀下限的國家特許電信商, 說遠傳主導是在秀下限喔, 一定會有貪小便宜秀下限想要..."
勿忘\w+\b,4,[]
\b\w+心酸的?\w*\b,78,"[不然問哪個是問心酸, 現在只有送心酸, 沒有5g看這不是看心酸, 辦心酸的喔, 5g網路都..."
\b\w+根本\w{2}\b,292,"[訊號根本沒用, 但是要出國跟那些重量級的打根本搞笑, 或是根本跑動, 但是回來後看售價根本..."
\b\w+敲盤子\w*\b,7,"[各家都拿高資費來敲盤子顆顆, 遠傳台哥是敲盤子喔, 根本敲盤子, 這合約分明敲盤子, 蘋果..."
\b\w+被當盤子\w*\b,58,"[我4不4被當盤子了, 到底是本人覺得被當盤子賺, 腳麻被當盤子, 只是被當盤子而已, 買舊..."


In [41]:
with open('matched_random.csv', 'w') as f:
    for key in matched.keys():
        f.write("%s,%s\n"%(key, matched[key]))